In [5]:
from openai import OpenAI
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

client = OpenAI()


In [6]:
train_file = client.files.create(
    file=open("/content/drive/MyDrive/train.jsonl", "rb"),
    purpose="fine-tune",   # 꼭 이걸로!
)

print("training_file id:", train_file.id)

training_file id: file-FYc91hYYt3NZoMr2quAiV2


In [7]:
job = client.fine_tuning.jobs.create(
    model="gpt-4.1-mini-2025-04-14",         # https://platform.openai.com/docs/guides/supervised-fine-tuning?upload=api
    training_file=train_file.id,
    method={
        "type": "supervised",
        "supervised": {
            "hyperparameters": {
                "n_epochs": 3,    # 데모용으로 epoch 수 명시 (안 쓰면 auto)
            },
        },
    },
    suffix="kr-support-category",   # (선택) 나중에 모델 이름에 붙을 suffix
)

print("job id:", job.id)
print("status:", job.status)


job id: ftjob-AVwcGj8U8K9MsbFhaebYWkDK
status: validating_files


In [9]:

job = client.fine_tuning.jobs.retrieve(job.id)
print(job.status, job.fine_tuned_model)


events = client.fine_tuning.jobs.list_events(
    fine_tuning_job_id=job.id,
    limit=5,
)
for e in events.data:
    print(e.message)


running None
Fine-tuning job started
Files validated, moving job to queued state
Validating training file: file-FYc91hYYt3NZoMr2quAiV2
Created fine-tuning job: ftjob-AVwcGj8U8K9MsbFhaebYWkDK


In [13]:
# finetuned_model = job.fine_tuned_model  # ft:... 문자열 그대로 붙여넣어도 됨

test_query = "받았는데 전원 스위치를 넣었는데 안 켜져요."

resp = client.responses.create(
    # model=finetuned_model,
    # model="ft:gpt-4.1-mini-2025-04-14:personal:kr-support-category:Cx6JT346",
    model = "gpt-4.1-mini-2025-04-14",
    input=test_query,
    instructions = "너는 고객 문의를 네 가지 카테고리 중 하나로만 분류한다: 배송, 환불, 제품, 기타. 다른 말은 하지 않는다."
)

print("모델 출력:", resp.output_text)

모델 출력: 제품


In [ ]:
{"messages": [
  {"role": "system", "content": "너는 고객 문의를 네 가지 카테고리 중 하나로만 분류한다: 배송, 환불, 제품, 기타. 다른 말은 하지 않는다."},
  {"role": "user", "content": "어제 주문한 상품이 아직 발송이 안됐어요. 언제 출고되나요?"},
  {"role": "assistant", "content": "배송"}
]}
